### Importação de bibliotecas

In [ ]:
import pandas as pd
import pathlib
import smtplib
from email.message import EmailMessage
from dotenv import load_dotenv
import os


load_dotenv()

### Importação do arquivo fonte de dados

In [26]:
vendas = pd.read_excel(r'Bases de Dados\Vendas.xlsx')
email = pd.read_excel(r'Bases de Dados\Emails.xlsx')
lojas = pd.read_csv(r'Bases de Dados\Lojas.csv', encoding = 'latin1', sep=';')


vendas = vendas.merge(lojas, on='ID Loja') #inserir o nome da loja na tabela vendas, conforme o ID da Loja.


vendas.info()
lojas.info()
email.info()

<class 'pandas.DataFrame'>
RangeIndex: 100999 entries, 0 to 100998
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Código Venda    100999 non-null  int64         
 1   Data            100999 non-null  datetime64[us]
 2   ID Loja         100999 non-null  int64         
 3   Produto         100999 non-null  str           
 4   Quantidade      100999 non-null  int64         
 5   Valor Unitário  100999 non-null  int64         
 6   Valor Final     100999 non-null  int64         
 7   Loja            100999 non-null  str           
dtypes: datetime64[us](1), int64(5), str(2)
memory usage: 6.2 MB
<class 'pandas.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   ID Loja  25 non-null     int64
 1   Loja     25 non-null     str  
dtypes: int64(1), str(1)
memory usage: 532.0 bytes
<class 'pandas.DataF

### Criação das tabelas de cada loja

In [27]:
dic_lojas = {}

for loja in lojas['Loja']:
    dic_lojas[loja] = vendas.loc[vendas['Loja'] == loja, :] # os ':" representam que quero todas as colunas que as linhas tenham a loja

dict_keys = dic_lojas.keys()

print(dict_keys)

dict_keys(['Iguatemi Esplanada', 'Shopping Midway Mall', 'Norte Shopping', 'Shopping Iguatemi Fortaleza', 'Shopping União de Osasco', 'Shopping Center Interlagos', 'Rio Mar Recife', 'Salvador Shopping', 'Rio Mar Shopping Fortaleza', 'Shopping Center Leste Aricanduva', 'Ribeirão Shopping', 'Shopping Morumbi', 'Parque Dom Pedro Shopping', 'Bourbon Shopping SP', 'Palladium Shopping Curitiba', 'Passei das Águas Shopping', 'Center Shopping Uberlândia', 'Shopping Recife', 'Shopping Vila Velha', 'Shopping SP Market', 'Shopping Eldorado', 'Shopping Ibirapuera', 'Novo Shopping Ribeirão Preto', 'Iguatemi Campinas', 'Shopping Barra'])


### Salvar os arquivos individuais

In [28]:
# identifica se a pasta existe, caso não, cria a pasta

caminho = pathlib.Path(r'Backup Arquivos Lojas')

data_salvar = vendas['Data'].max()

arquivo_pasta = caminho.iterdir()
nome_arquivos = [arquivo.name for arquivo in arquivo_pasta]

for loja in dic_lojas:
    if loja not in nome_arquivos:
        pasta = caminho / loja
        pasta.mkdir()

# salva o arquivo dentro da pasta com o nome da loja

arquivo = '{}_{}_{}.xlsx'.format(data_salvar.month, data_salvar.day, loja) # gerou o nome do arquivo
local = caminho / loja / arquivo # determina o local para salvar o arquivo 

dic_lojas[loja].to_excel(local) # criou os arquivos de cada loja, dentro das pastas de cada loja


### Estabelecer as metas

In [29]:
meta_faturamento_ano = 1650000
meta_faturamento_dia = 1000
meta_produtos_ano =120  # (quantos produtos diferentes foram vendidos naquele período)
meta_produtos_dia =4
meta_ticket_ano = 500
meta_ticket_dia = 500

### Determinar os indicadores para cada loja e Enviar Email com os One Page para cada Loja

In [ ]:
for loja in dic_lojas:
    dados_loja = dic_lojas[loja]
    venda_loja_ano = dados_loja['Valor Final'].sum()
    venda_loja_dia = dados_loja.loc[dados_loja['Data']==data_salvar]['Valor Final'].sum()
    produto_loja_ano = len(dados_loja['Produto'].unique())
    produto_loja_dia = len(dados_loja.loc[dados_loja['Data']==data_salvar]['Produto'].unique())
    valor_por_venda = dados_loja.groupby('Código Venda').sum(numeric_only=True)
    valor_por_venda_dia = dados_loja.loc[dados_loja['Data']==data_salvar].groupby('Código Venda').sum(numeric_only=True)
    ticket_loja_ano = valor_por_venda['Valor Final'].mean()
    ticket_loja_dia = valor_por_venda_dia['Valor Final'].mean()
    
    if venda_loja_ano >= meta_faturamento_ano:
        cor_fat_ano = "green"
    else:
        cor_fat_ano = "red"
    
    if venda_loja_dia >= meta_faturamento_dia:
        cor_fat_dia = "green"
    else:
        cor_fat_dia = "red"
    
    if ticket_loja_ano >= meta_ticket_ano:
        cor_tichet_ano = "green" 
    else:
        cor_ticket_ano = "red"
    
    if ticket_loja_dia >= meta_ticket_dia:
        cor_ticket_dia = "green"
    else: 
        cor_ticket_dia = "red"
    
    if produto_loja_ano >= meta_produtos_ano:
        cor_produto_ano = "green"
    else: 
        cor_produto_ano = "red"
    
    if produto_loja_dia >= meta_produtos_dia:
        cor_produto_dia = "green" 
    else: 
        cor_produto_dia = "red"


## Enviando os emails

    SENHA_GMAIL=os.getenv('SENHA_GMAIL')

    def enviar_email():
        msg = EmailMessage()
        msg['Subject'] = "One Page"
        msg['From'] ='conta2trader@gmail.com'
        msg['To'] ='andremontagna@gmail.com'
        msg['Cc']
        msg['BCo']

        mensagem = f"""
        <p>O Resultado na data de <strong>{data_salvar.day} / {data_salvar.month}</strong> da <strong>Loja {loja}</strong> foi de:</p>
    
        <table>
        <tr>
            <th>Indicador</th>
            <th style="text-align: center">Valor do Dia</th>
            <th style="text-align: center">Meta do Dia</th>
            <th style="text-align: center">Cenário do Dia</th>
        </tr>
        <tr>
            <td>Faturamento</td>
            <td style="text-align: center"> R$ {venda_loja_dia: .2f}</td>
            <td style="text-align: center">R$ {meta_faturamento_dia: .2f}</td>
            <td style="text-align: center"><font color = {cor_fat_dia}>◙</font></td>
        </tr>
        <tr>
            <td>Ticket Médio</td>
            <td style="text-align: center">R$ {ticket_loja_dia: .2f}</td>
            <td style="text-align: center">R$ {meta_ticket_dia: .2f}</td>
            <td style="text-align: center"><font color = {cor_ticket_dia}>◙</font></td>
        </tr>
        <tr>
            <td>Diversidade de Produtos</td>
            <td style="text-align: center">{produto_loja_dia}</td>
            <td style="text-align: center">{meta_produtos_dia}</td>
            <td style="text-align: center"><font color = {cor_produto_dia}>◙</font></td>
        </tr>
        </table>

        <br>

        <table>
        <tr>
            <th>Indicador</th>
            <th style="text-align: center">Valor do Ano</th>
            <th style="text-align: center">Meta do Ano</th>
            <th style="text-align: center">Cenário do Ano</th>
        </tr>
        <tr>
            <td>Faturamento</td>
            <td style="text-align: center"> R$ {venda_loja_ano: .2f}</td>
            <td style="text-align: center">R$ {meta_faturamento_ano: .2f}</td>
            <td style="text-align: center"><font color = {cor_fat_ano}>◙</font></td>
        </tr>
        <tr>
            <td>Ticket Médio</td>
            <td style="text-align: center">R$ {ticket_loja_ano: .2f}</td>
            <td style="text-align: center">R$ {meta_ticket_ano: .2f}</td>
            <td style="text-align: center"><font color = {cor_tichet_ano}>◙</font></td>
        </tr>
        <tr>
            <td>Diversidade de Produtos</td>
            <td style="text-align: center">{produto_loja_ano}</td>
            <td style="text-align: center">{meta_produtos_ano}</td>
            <td style="text-align: center"><font color = {cor_produto_ano}>◙</font></td>
        </tr>
        </table>
        <br>

        <p> Segue anexo a planilha detalhada para análise</p>

        """
        mensagem = mensagem.encode('utf-8')

        msg.add_header('Content-Type', 'text/html')

        msg.set_payload(mensagem)

        try:
            with smtplib.SMTP('smtp.gmail.com', 587) as servidor:
                servidor.starttls()
                servidor.login(msg['From'],SENHA_GMAIL)
                servidor.send_message(msg)
                print("E-mail enviado com sucesso!")
        except Exception as e:
            print(f"Erro ao enviar: {e}")
    
    
    enviar_email()


E-mail enviado com sucesso!
E-mail enviado com sucesso!
E-mail enviado com sucesso!
E-mail enviado com sucesso!
E-mail enviado com sucesso!
E-mail enviado com sucesso!
E-mail enviado com sucesso!
E-mail enviado com sucesso!
E-mail enviado com sucesso!
E-mail enviado com sucesso!
E-mail enviado com sucesso!
E-mail enviado com sucesso!
E-mail enviado com sucesso!
E-mail enviado com sucesso!
E-mail enviado com sucesso!
E-mail enviado com sucesso!
E-mail enviado com sucesso!
E-mail enviado com sucesso!
E-mail enviado com sucesso!
E-mail enviado com sucesso!
E-mail enviado com sucesso!
E-mail enviado com sucesso!
E-mail enviado com sucesso!
E-mail enviado com sucesso!
E-mail enviado com sucesso!


### Enviar Email